In [137]:
!pip3 install scikit-learn
!pip3 install imageio
!pip3 install IPython

import imageio
from IPython.display import Image
import json
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
import tensorflow.python.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from numpy import save, load

from sklearn.metrics import (
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)

In [138]:
import os

# tensorflow.keras.backend.set_floatx('float16')

PATCH_SIZE_X=32
PATCH_SIZE_Y=32
PATCH_SIZE_Z=32

SampleX = []
SampleY = []
LabelY = []

category_1_count = 0
category_2_count = 0
category_3_count = 0
category_4_count = 0
total_count = 0

def weighted_categorical_crossentropy(weights):
    weights = K.variable(weights)        
    def loss(y_true, y_pred):
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss    
    return loss

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (
                K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
                    
def soft_dice_loss(y_true, y_pred):
    print("y_true")
    print(y_true.shape)
    print("y_pred")
    print(y_pred.shape)
    dice_loss = 1 - (1/4 * (dice_coef(y_true[0][0],y_pred[0][0])+dice_coef(y_true[0][1],y_pred[0][1])+dice_coef(y_true[0][2],y_pred[0][2])+dice_coef(y_true[0][3],y_pred[0][3])))
    # dice_loss = 1 - (1/3 * (dice_coef(y_true[0][0],y_pred[0][0])+dice_coef(y_true[0][1],y_pred[0][1])+dice_coef(y_true[0][2],y_pred[0][2])))
    return dice_loss

datasets=['4channel-5/']

# datasets=['model1batch1/']
# datasets=['model1batch2/'] - best 
# datasets=['model1batch3/']
# datasets=['model1batch4/'] 
# datasets=['modl1batch5/'] - worst
# datasets=['model1batch6/']
# datasets=['model1batch7/']
# datasets = ['model1batch8/']
# datasets = ['model1batch9/']
# datasets=['model1batch10/']
# datasets=['model1batch11/']

for dataset in datasets:
    for patch_file in os.listdir('../input/'+dataset):
        if "nifti" in patch_file:
            image = np.load('../input/'+dataset+patch_file)
            train_x = np.empty((4,PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
            patient_id = patch_file.split('-')[1]
            patch_num = patch_file.split('-')[2].split(".")[0]
            ngtdm = 0
            for x in range(0,PATCH_SIZE_X):
                for y in range(0,PATCH_SIZE_Y):
                    for z in range(0,PATCH_SIZE_Z):
                        train_x[0][x][y][z] = np.uint8(image[0][x][y][z])
                        train_x[1][x][y][z] = np.uint8(image[1][x][y][z])
                        train_x[2][x][y][z] = image[2][x][y][z]
                        train_x[3][x][y][z] = image[3][x][y][z]
                        total_count = total_count + 1
            SampleX.append(train_x)
            # print('image shape')
            # print(image.shape)
        
        if "label" in patch_file:
            label = np.load('../input/'+dataset+patch_file)
            train_y = np.empty((4,PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
            patient_id = patch_file.split('-')[1]
            patch_num = patch_file.split('-')[2].split(".")[0]
            for x in range(0,PATCH_SIZE_X):
                for y in range(0,PATCH_SIZE_Y):
                    for z in range(0,PATCH_SIZE_Z):
                        if label[x][y][z] == 1:
                            train_y[0][x][y][z] = 1
                            train_y[1][x][y][z] = 0
                            train_y[2][x][y][z] = 0
                            train_y[3][x][y][z] = 0
                            category_1_count = category_1_count + 1
                        elif label[x][y][z] == 2:
                            # print("category (%d,%d,%d) 1" % (x,y,z))
                            train_y[0][x][y][z] = 0
                            train_y[1][x][y][z] = 1
                            train_y[2][x][y][z] = 0
                            train_y[3][x][y][z] = 0
                            category_2_count = category_2_count + 1
                        elif label[x][y][z] == 4:
                            # print("category (%d,%d,%d) 2" % (x,y,z))
                            train_y[0][x][y][z] = 0
                            train_y[1][x][y][z] = 0
                            train_y[2][x][y][z] = 0
                            train_y[3][x][y][z] = 1
                            category_4_count = category_4_count + 1
                        else:
                            # print("category (%d,%d,%d) 3" % (x,y,z))
                            train_y[0][x][y][z] = 0
                            train_y[1][x][y][z] = 0
                            train_y[2][x][y][z] = 1
                            train_y[3][x][y][z] = 0
                            category_3_count = category_3_count + 1
            LabelY.append(label)
            SampleY.append(train_y)
            # print('image shape')
            # print(image.shape)

''' TBU
weight_1 = total_count/(4*category_1_count)
weight_2 = total_count/(4*category_2_count)
weight_3 = total_count/(4*category_3_count)
weight_4 = total_count/(4*category_4_count)
print("weight-1: %f" % (weight_1))
print("weight-2: %f" % (weight_2))
print("weight-3: %f" % (weight_3))
print("weight-4: %f" % (weight_4))
'''

print("category_1_count: %d" % (category_1_count))
print("category-2-count: %d" % (category_2_count))
print("category-3-count: %d" % (category_3_count))
print("category-4-count: %d" % (category_4_count))
print("total count: %d" % (total_count))

''' TBU Save-Retrain-Save Loop
weights = np.empty((4,32,32,32))
for x in range(0,32):
    for y in range(0,32):
        for z in range(0,32):
            weights[0][x][y][z] = weight_1
            weights[1][x][y][z] = weight_2
            weights[2][x][y][z] = weight_3
            weights[3][x][y][z] = weight_4
TBU ''' 

from keras.utils.np_utils import to_categorical
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

''' TBU Save + reload '''
num_patches=len(SampleX)
n1,n2,n3,n4 = SampleX[0].shape
print("Sample: %d,%d,%d,%d" % (n1,n2,n3,n4))
InputX = np.array(SampleX)
InputX = InputX.reshape((num_patches,n1*n2*n3*n4))
print("SampleY shape: ")
print(LabelY[0].shape)
n1,n2,n3 = LabelY[0].shape
print("Sample: %d,%d,%d" % (n1,n2,n3))
InputY = np.array(LabelY)
InputY = InputY.reshape((num_patches,n1*n2*n3))
''' TBU Save + reload '''

''' TBU: Save + Reload ''' 
model=tensorflow.keras.models.load_model('../input/savedmodel/segmentation.model',custom_objects={'soft_dice_loss':soft_dice_loss})
model.save('./segmentation.model')

# Load + Retrain
iteration = 0
kfold = KFold(n_splits=5, shuffle=True)
for train, test in kfold.split(InputX, InputY):
    trainX = np.array(SampleX)[train]
    trainY = np.array(SampleY)[train]
    validationX = np.array(SampleX)[test]
    validationY = np.array(SampleY)[test]
    
    print("Model fitting iteration %d started" % (iteration))
    # model.fit(trainX,trainY,batch_size=1,epochs=16)
    
    model=tensorflow.keras.models.load_model('./segmentation.model',custom_objects={'soft_dice_loss':soft_dice_loss})
    model.fit(np.array(trainX),np.array(trainY),batch_size=1,epochs=16,shuffle=True)
    
    print("model fit done")
    print(model.metrics)
    print("saving model")
    model.save('./segmentation.model')
    
    print("model save done")
    print("Model fitting iteration %d done" % (iteration))
    iteration = iteration + 1

    probY = model.predict(np.array(validationX))
    print("probY shape: ")
    print(probY.shape)
    
    probY = to_categorical(np.argmax(probY,axis=1), 4)
    print("probY shape: ")
    print(probY.shape)
    
    predY = probY.tolist()
    # print(predY)

    width=PATCH_SIZE_X
    height=PATCH_SIZE_Y
    depth=PATCH_SIZE_Z

    Y_True = []
    for v in validationY:
        for x in range(0,width):
            for y in range(0,height):
                for z in range(0,depth):
                    for i in range(0,4):
                        if (v[i][x][y][z] == 1):
                            Y_True.append(i)
                            break

    Y_Pred = []
    for p in predY:
        for x in range(0,width):
            for y in range(0,height):
                for z in range(0,depth):
                    for i in range(0,4):
                        if (p[x][y][z][i] == 1):
                            Y_Pred.append(i)
                            break
    
    print("Confusion Matrix: ")
    print(multilabel_confusion_matrix(np.array(Y_True), np.array(Y_Pred)))

    print('Precision, Recall, F1 Score: ')
    # print(f1_score(np.array(Y_True), np.array(Y_Pred)))
    print(precision_recall_fscore_support(np.array(Y_True), np.array(Y_Pred), average='weighted', labels=np.unique(Y_Pred)))

    # model=tensorflow.keras.models.load_model('./segmentation.model',custom_objects={'soft_dice_loss':soft_dice_loss})
    
'''TBU Save-Retrain-Save Loop'''

''' TBU: First iteration
# Iterate and plot random images
for i in range(9):
    img = np.empty((PATCH_SIZE_X,PATCH_SIZE_Y))
    for y in range(0,PATCH_SIZE_X):
        for z in range(0,PATCH_SIZE_Y):
            img[y][z] = train_x[0][i][y][z]
    plt.subplot(3, 3, i + 1)
    plt.imshow(img, cmap='gray')
    plt.axis('off')

#weight_1 = total_count/(4*category_1_count)
#weight_2 = total_count/(4*category_2_count)
#weight_3 = total_count/(4*category_3_count)
#weight_4 = total_count/(4*category_4_count)

print("Total count: %d" % (total_count))
print("Category-1 count: %d" % category_1_count)
print("Category-2 count: %d" % category_2_count)
print("Category-3 count: %d" % category_3_count)
print("Category-4 count: %d" % category_4_count)

TBU: First iteration '''

category_1_count: 43168
category-2-count: 285143
category-3-count: 12686452
category-4-count: 92437
total count: 13107200
Sample: 4,32,32,32
SampleY shape: 
(32, 32, 32)
Sample: 32,32,32
Model fitting iteration 0 started
Epoch 1/16
y_true
(1, 4, 32, 32, 32)
y_pred
(1, 4, 32, 32, 32)
y_true
(1, 4, 32, 32, 32)
y_pred
(1, 4, 32, 32, 32)
320/320 [==============================] - 12s 35ms/step - loss: 0.0768 - mean_io_u: 0.9672
Epoch 2/16
320/320 [==============================] - 11s 35ms/step - loss: 0.0768 - mean_io_u: 0.9672
Epoch 3/16
320/320 [==============================] - 11s 35ms/step - loss: 0.0768 - mean_io_u: 0.9672
Epoch 4/16
320/320 [==============================] - 11s 35ms/step - loss: 0.0768 - mean_io_u: 0.9672
Epoch 5/16
320/320 [==============================] - 11s 35ms/step - loss: 0.0768 - mean_io_u: 0.9672
Epoch 6/16
320/320 [==============================] - 11s 35ms/step - loss: 0.0768 - mean_io_u: 0.9672
Epoch 7/16
320/320 [==============================] - 11s

' TBU: First iteration\n# Iterate and plot random images\nfor i in range(9):\n    img = np.empty((PATCH_SIZE_X,PATCH_SIZE_Y))\n    for y in range(0,PATCH_SIZE_X):\n        for z in range(0,PATCH_SIZE_Y):\n            img[y][z] = train_x[0][i][y][z]\n    plt.subplot(3, 3, i + 1)\n    plt.imshow(img, cmap=\'gray\')\n    plt.axis(\'off\')\n\n#weight_1 = total_count/(4*category_1_count)\n#weight_2 = total_count/(4*category_2_count)\n#weight_3 = total_count/(4*category_3_count)\n#weight_4 = total_count/(4*category_4_count)\n\nprint("Total count: %d" % (total_count))\nprint("Category-1 count: %d" % category_1_count)\nprint("Category-2 count: %d" % category_2_count)\nprint("Category-3 count: %d" % category_3_count)\nprint("Category-4 count: %d" % category_4_count)\n\nTBU: First iteration '

In [139]:
''' TBU: First iteration
import tensorflow
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose
from tensorflow.keras.optimizers import Adam
from keras.layers.merge import concatenate

keras.backend.set_image_data_format("channels_first")
TBU: First iteration '''

' TBU: First iteration\nimport tensorflow\nfrom tensorflow import keras\nfrom keras import backend as K\nfrom tensorflow.keras.layers import Input\nfrom tensorflow.keras.models import Model\nfrom tensorflow.keras.layers import Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, PReLU, Conv3DTranspose\nfrom tensorflow.keras.optimizers import Adam\nfrom keras.layers.merge import concatenate\n\nkeras.backend.set_image_data_format("channels_first")\nTBU: First iteration '

In [140]:
''' TBU: First iteration
input_layer = Input(shape=(4,32,32,32))
# input_layer = Input(shape=(3,32,32,32))
# input_layer = Input(shape=(1,32,32,32))
input_layer

down_depth_0_layer_0 = keras.layers.Conv3D(filters=32, 
                              kernel_size=(3,3,3),
                              padding='same',
                              strides=(1,1,1),
                              data_format='channels_first',
                              input_shape=(4,32,32,32)
                              # input_shape=(1,32,32,32)
                              )(input_layer)

down_depth_0_layer_0
TBU: First iteration '''

" TBU: First iteration\ninput_layer = Input(shape=(4,32,32,32))\n# input_layer = Input(shape=(3,32,32,32))\n# input_layer = Input(shape=(1,32,32,32))\ninput_layer\n\ndown_depth_0_layer_0 = keras.layers.Conv3D(filters=32, \n                              kernel_size=(3,3,3),\n                              padding='same',\n                              strides=(1,1,1),\n                              data_format='channels_first',\n                              input_shape=(4,32,32,32)\n                              # input_shape=(1,32,32,32)\n                              )(input_layer)\n\ndown_depth_0_layer_0\nTBU: First iteration "

In [141]:
''' TBU: First iteration
down_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)
down_depth_0_layer_0
TBU First iteration '''

" TBU: First iteration\ndown_depth_0_layer_0 = Activation('relu')(down_depth_0_layer_0)\ndown_depth_0_layer_0\nTBU First iteration "

In [142]:
''' TBU: First iteration
# Create a Conv3D layer with 64 filters and add relu activation
down_depth_0_layer_1 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_0)
down_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)
down_depth_0_layer_1
TBU First iteration '''


" TBU: First iteration\n# Create a Conv3D layer with 64 filters and add relu activation\ndown_depth_0_layer_1 = Conv3D(filters=64, \n                kernel_size=(3,3,3),\n                padding='same',\n                strides=(1,1,1)\n               )(down_depth_0_layer_0)\ndown_depth_0_layer_1 = Activation('relu')(down_depth_0_layer_1)\ndown_depth_0_layer_1\nTBU First iteration "

In [143]:
''' TBU: First iteration
# Define a max pooling layer
# down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)
down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2),data_format='channels_first')(down_depth_0_layer_1)
down_depth_0_layer_pool
TBU: First iteration '''

" TBU: First iteration\n# Define a max pooling layer\n# down_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2))(down_depth_0_layer_1)\ndown_depth_0_layer_pool = MaxPooling3D(pool_size=(2,2,2),data_format='channels_first')(down_depth_0_layer_1)\ndown_depth_0_layer_pool\nTBU: First iteration "

In [144]:
''' TBU: First iteration
# Add a Conv3D layer to your network with relu activation
down_depth_1_layer_0 = Conv3D(filters=64, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_0_layer_pool)
down_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)
down_depth_1_layer_0
TBU: First iteration '''

" TBU: First iteration\n# Add a Conv3D layer to your network with relu activation\ndown_depth_1_layer_0 = Conv3D(filters=64, \n                kernel_size=(3,3,3),\n                padding='same',\n                strides=(1,1,1)\n               )(down_depth_0_layer_pool)\ndown_depth_1_layer_0 = Activation('relu')(down_depth_1_layer_0)\ndown_depth_1_layer_0\nTBU: First iteration "

In [145]:
''' TBU: First iteration
# Add another Conv3D with 128 filters to your network.
down_depth_1_layer_1 = Conv3D(filters=128, 
                kernel_size=(3,3,3),
                padding='same',
                strides=(1,1,1)
               )(down_depth_1_layer_0)
down_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)
down_depth_1_layer_1
TBU: First iteration '''

" TBU: First iteration\n# Add another Conv3D with 128 filters to your network.\ndown_depth_1_layer_1 = Conv3D(filters=128, \n                kernel_size=(3,3,3),\n                padding='same',\n                strides=(1,1,1)\n               )(down_depth_1_layer_0)\ndown_depth_1_layer_1 = Activation('relu')(down_depth_1_layer_1)\ndown_depth_1_layer_1\nTBU: First iteration "

In [146]:
''' TBU: First iteration
# Add an upsampling operation to your network
up_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)
up_depth_0_layer_0
TBU: First iteration '''

' TBU: First iteration\n# Add an upsampling operation to your network\nup_depth_0_layer_0 = UpSampling3D(size=(2,2,2))(down_depth_1_layer_1)\nup_depth_0_layer_0\nTBU: First iteration '

In [147]:
''' TBU: First iteration
# Print the shape of layers to concatenate
print(up_depth_0_layer_0)
print()
print(down_depth_0_layer_1)
TBU: First iteration '''

' TBU: First iteration\n# Print the shape of layers to concatenate\nprint(up_depth_0_layer_0)\nprint()\nprint(down_depth_0_layer_1)\nTBU: First iteration '

In [148]:
''' TBU: First iteration
# Add a concatenation along axis 1
up_depth_1_concat = concatenate([up_depth_0_layer_0,
                                 down_depth_0_layer_1],
                                axis=1)
up_depth_1_concat
TBU: First iteration '''

' TBU: First iteration\n# Add a concatenation along axis 1\nup_depth_1_concat = concatenate([up_depth_0_layer_0,\n                                 down_depth_0_layer_1],\n                                axis=1)\nup_depth_1_concat\nTBU: First iteration '

In [149]:
''' TBU: First iteration
down_depth_0_layer_1
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_1 = keras.layers.Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_concat)
up_depth_1_layer_1 = keras.layers.Activation('relu')(up_depth_1_layer_1)
up_depth_1_layer_1
TBU: First iteration '''

" TBU: First iteration\ndown_depth_0_layer_1\n# Add a Conv3D up-convolution with 64 filters to your network\nup_depth_1_layer_1 = keras.layers.Conv3D(filters=64, \n                            kernel_size=(3,3,3),\n                            padding='same',\n                            strides=(1,1,1)\n                           )(up_depth_1_concat)\nup_depth_1_layer_1 = keras.layers.Activation('relu')(up_depth_1_layer_1)\nup_depth_1_layer_1\nTBU: First iteration "

In [150]:
''' TBU: First iteration
# Add a Conv3D up-convolution with 64 filters to your network
up_depth_1_layer_2 = keras.layers.Conv3D(filters=64, 
                            kernel_size=(3,3,3),
                            padding='same',
                            strides=(1,1,1)
                           )(up_depth_1_layer_1)
up_depth_1_layer_2 = keras.layers.Activation('relu')(up_depth_1_layer_2)
up_depth_1_layer_2
TBU: First iteration '''

" TBU: First iteration\n# Add a Conv3D up-convolution with 64 filters to your network\nup_depth_1_layer_2 = keras.layers.Conv3D(filters=64, \n                            kernel_size=(3,3,3),\n                            padding='same',\n                            strides=(1,1,1)\n                           )(up_depth_1_layer_1)\nup_depth_1_layer_2 = keras.layers.Activation('relu')(up_depth_1_layer_2)\nup_depth_1_layer_2\nTBU: First iteration "

In [151]:
''' TBU: First iteration
# Add a final Conv3D with 3 filters to your network.
final_conv = keras.layers.Conv3D(filters=4, #4 categories 
                    kernel_size=(1,1,1),
                    padding='valid',
                    strides=(1,1,1)
                    )(up_depth_1_layer_2)
final_conv
TBU: First iteration '''

" TBU: First iteration\n# Add a final Conv3D with 3 filters to your network.\nfinal_conv = keras.layers.Conv3D(filters=4, #4 categories \n                    kernel_size=(1,1,1),\n                    padding='valid',\n                    strides=(1,1,1)\n                    )(up_depth_1_layer_2)\nfinal_conv\nTBU: First iteration "

In [152]:
''' TBU: First iteration
# Add a sigmoid activation to your final convolution.
final_activation = keras.layers.Activation('sigmoid')(final_conv)
final_activation
TBU: First iteration '''

" TBU: First iteration\n# Add a sigmoid activation to your final convolution.\nfinal_activation = keras.layers.Activation('sigmoid')(final_conv)\nfinal_activation\nTBU: First iteration "

In [153]:
''' TBU: First iteration
def weighted_categorical_crossentropy(weights):
    weights = K.variable(weights)        
    def loss(y_true, y_pred):
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    return loss

def plot_roc_curve(fpr,tpr): 
    plt.plot(fpr,tpr) 
    plt.axis([0,1,0,1]) 
    plt.xlabel('False Positive Rate') 
    plt.ylabel('True Positive Rate') 
    plt.show()    
  
model = tensorflow.keras.models.Model(inputs=input_layer, outputs=final_activation)

weights = np.empty((4,32,32,32))
for x in range(0,32):
    for y in range(0,32):
        for z in range(0,32):
            weights[0][x][y][z] = weight_1
            weights[1][x][y][z] = weight_2
            weights[2][x][y][z] = weight_3
            weights[3][x][y][z] = weight_4

model.compile(optimizer=tensorflow.keras.optimizers.Adam(lr=0.00001),
              # loss='categorical_crossentropy',
              # loss=weighted_categorical_crossentropy(weights),
              loss=soft_dice_loss,
              # metrics=['categorical_accuracy']
              metrics=[tensorflow.keras.metrics.MeanIoU(num_classes=4)]
             )
model.summary()

num_patches=len(SampleX)
print("num_patches: %d" % num_patches)
n1,n2,n3,n4 = SampleX[0].shape
print("Sample: %d,%d,%d,%d" % (n1,n2,n3,n4))
InputX = np.array(SampleX)
InputX = InputX.reshape((num_patches,n1*n2*n3*n4))
print("SampleY shape: ")
print("num patches: %d" % len(LabelY))
print(LabelY[0].shape)
n1,n2,n3 = LabelY[0].shape
print("Sample: %d,%d,%d" % (n1,n2,n3))
InputY = np.array(LabelY)
InputY = InputY.reshape((num_patches,n1*n2*n3))

iteration = 0
kfold = KFold(n_splits=5, shuffle=True)
for train, test in kfold.split(InputX, InputY):
    trainX = np.array(SampleX)[train]
    trainY = np.array(SampleY)[train]
    validationX = np.array(SampleX)[test]
    validationY = np.array(SampleY)[test]
    print("Model fitting iteration %d started" % (iteration))
    model.fit(trainX,trainY,batch_size=1,epochs=16)
    # model.fit(trainX,trainY,batch_size=1,epochs=1)
    print("Model fitting iteration %d done" % (iteration))
    iteration = iteration + 1

from keras.utils.np_utils import to_categorical

probY = model.predict(np.array(validationX))
print("probY shape: ")
print(probY.shape)
probY = to_categorical(np.argmax(probY,axis=1), 4)
print("probY shape: ")
print(probY.shape)
predY = probY.tolist()
# print(predY)

from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

width=PATCH_SIZE_X
height=PATCH_SIZE_Y
depth=PATCH_SIZE_Z

Y_True = []
for v in validationY:
    for x in range(0,width):
        for y in range(0,height):
            for z in range(0,depth):
                for i in range(0,4):
                    if (v[i][x][y][z] == 1):
                        Y_True.append(i)
                        break

Y_Pred = []
for p in predY:
    for x in range(0,width):
        for y in range(0,height):
            for z in range(0,depth):
                for i in range(0,4):
                    if (p[x][y][z][i] == 1):
                        Y_Pred.append(i)
                        # if (i != 0):
                        #    print("Pred: %d,%d,%d: label: %d" % (x,y,z,i))
                        break
    
print("Confusion Matrix: ")
print(multilabel_confusion_matrix(np.array(Y_True), np.array(Y_Pred)))

print('Precision, Recall, F1 Score: ')
# print(f1_score(np.array(Y_True), np.array(Y_Pred)))
print(precision_recall_fscore_support(np.array(Y_True), np.array(Y_Pred), average='weighted', labels=np.unique(Y_Pred)))
TBU '''

'''
TestX = []
PatchFileNames = []
def load_dicom_files():
    datasets=['../input/segmentation-model-input-2/']
    # datasets=['../input/dicom4/']
    # datasets=['../input/dicom1/', '../input/dicoom2/', '../input/dicom3', '../input/dicom4']
    for dataset in datasets:
        for patch_file in os.listdir(dataset):
            # TBU if "dicom" in patch_file:
            if "nifti" in patch_file:
                PatchFileNames.append(patch_file)
                image = np.load(dataset+patch_file)
                test_x = np.empty((3,PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
                patient_id = patch_file.split('-')[1]
                patch_num = patch_file.split('-')[2].split(".")[0]
                ngtdm = 0
                for x in range(0,PATCH_SIZE_X):
                    for y in range(0,PATCH_SIZE_Y):
                        for z in range(0,PATCH_SIZE_Z):
                            test_x[0][x][y][z] = np.uint8(image[0][x][y][z])
                            test_x[1][x][y][z] = np.uint8(image[1][x][y][z])
                            test_x[2][x][y][z] = image[3][x][y][z]
                TestX.append(test_x)

load_dicom_files()
probY = model.predict(np.array(TestX))
print("probY shape: ")
print(probY.shape)

probY = to_categorical(np.argmax(probY,axis=1), 4)
print("probY shape: ")
print(probY.shape)

# predY = probY.tolist()
# print(predY)

patch_count = 0
for y_pred in probY:
    width,height,depth,channels = y_pred.shape
    valid_patch_found = False
    count_pixels_with_label_1 = 0
    count_pixels_with_label_2 = 0
    count_pixels_with_label_3 = 0
    count_pixels_with_label_4 = 0
    for x in range(0,width):
        for y in range(0,height):
            for z in range(0,depth):
                if y_pred[x][y][z][0] == 1:
                    count_pixels_with_label_1 = count_pixels_with_label_1 + 1
                elif y_pred[x][y][z][1] == 1:
                    count_pixels_with_label_2 = count_pixels_with_label_2 + 1 
                elif y_pred[x][y][z][2] == 1:
                    count_pixels_with_label_3 = count_pixels_with_label_3 + 1
                else:
                    count_pixels_with_label_4 = count_pixels_with_label_4 + 1
    print("Count of non-background pixels %d" % (count_pixels_with_label_1+count_pixels_with_label_2+count_pixels_with_label_4))
    # TBU: if ((count_pixels_with_label_1+count_pixels_with_label_2+count_pixels_with_label_4)/(32*32*32) > .5):
    valid_patch = np.empty((32,32,32,2))
    for x in range(0,32):
        for y in range(0,32):
            for z in range(0,32):
                valid_patch[x][y][z][0] = TestX[patch_count][0][x][y][z]
                if y_pred[x][y][z][0] == 1:
                    valid_patch[x][y][z][1] = 1
                elif y_pred[x][y][z][1] == 1:
                    valid_patch[x][y][z][1] = 2
                elif y_pred[x][y][z][3] == 1:
                    valid_patch[x][y][z][1] = 4
                else:
                    valid_patch[x][y][z][1] = 3
    # output_file = './' + 'densenet-input-' + PatchFileNames[patch_count]
    output_file = './' + 'segmented-image-' + PatchFileNames[patch_count]
    print('Picked %d, %s for training densenet' % (patch_count, output_file))
    save(output_file, valid_patch)
    patch_count = patch_count + 1

# filter only slices with labels 1,2,4
# Create a patch array with 32x32x32 by 2 (greyscale+label)
# Save these slices to output directory

# Download them 
# Create label patch array with 32x32x32 by 1 for MGMT label
# Train densenet model 
# get confusion matrix
# get f1 score, precision, recall

fpr = dict()
tpr = dict()
for i in range(0,4):
    Vy = []
    # print("vy: ")
    for v in validationY:
        width, height, depth = v[i].shape
        vy = np.empty((width,height,depth))
        for x in range(0,width):
            for y in range(0,height):
                for z in range(0,depth):
                    vy[x][y][z] = v[i][x][y][z]
        flattened_vy = vy.flatten()
        # print(flattened_vy)
        Vy.extend(flattened_vy.tolist())
    # print('Vy: ')
    # print(Vy)
    Py = []
    # print("py: ")
    for p in predY:
        width = height = depth = len(p[i])
        print('width %d, height %d, depth %d' % (width, height, depth))
        py = np.empty((width,height,depth))
        for x in range(0,width):
            for y in range(0,height):
                for z in range(0,depth):
                    # print('i %d, x %d, y %d, z %d' % (i, x, y, z))
                    py[x][y][z] = p[x][y][z][i]
        flattened_py = py.flatten()
        # print(flattened_py)
        Py.extend(flattened_py.tolist())
    # print('Py: ')
    # print(Py)
    # print("V: class %d" % i)
    for v in Vy:
        if ((v != 0) and (v != 1)):
            # print("v: %d" % v)
            break
    # print("P: class %d" % i)
    for p in Py:
        if ((p != 0) and (p != 1)):
            # print("p: %d" % p)
            break
    fpr[i], tpr[i], _ = roc_curve(np.array(Vy), np.array(Py))

for i in range(0,4):
    plot_roc_curve(fpr[i],tpr[i]) 

print("model fit done")
print(model.metrics)
print("saving model")
model.save('./segmentation.model')
print("model save done") '''

'\nTestX = []\nPatchFileNames = []\ndef load_dicom_files():\n    datasets=[\'../input/segmentation-model-input-2/\']\n    # datasets=[\'../input/dicom4/\']\n    # datasets=[\'../input/dicom1/\', \'../input/dicoom2/\', \'../input/dicom3\', \'../input/dicom4\']\n    for dataset in datasets:\n        for patch_file in os.listdir(dataset):\n            # TBU if "dicom" in patch_file:\n            if "nifti" in patch_file:\n                PatchFileNames.append(patch_file)\n                image = np.load(dataset+patch_file)\n                test_x = np.empty((3,PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))\n                patient_id = patch_file.split(\'-\')[1]\n                patch_num = patch_file.split(\'-\')[2].split(".")[0]\n                ngtdm = 0\n                for x in range(0,PATCH_SIZE_X):\n                    for y in range(0,PATCH_SIZE_Y):\n                        for z in range(0,PATCH_SIZE_Z):\n                            test_x[0][x][y][z] = np.uint8(image[0][x][y][z])\n   